In [0]:
%run ./Designer

In [0]:
class Ingestion():
    def __init__(
        self
        ,FeedTopics_file: str
    ):
        self.FeedTopics = read_yaml(FeedTopics_file)
        self.dict_Feed = self.get_Feed()
        self.topics = pd.DataFrame(columns=['topic_name','topic_schema','topic_BronzeTblName'])
        self.list_of_topics = ['books','customers','orders']

    def get_Feed(self)-> dict:
        FeedTopics = self.FeedTopics
        return {
            "feed_path": FeedTopics['feed']['path']
            ,"feed_schema": FeedTopics['feed']['schema']
        }

    def get_Topics(self):
        FeedTopics = self.FeedTopics
        topics = FeedTopics['topics']
        for topic in topics:
            topic_name = topic['name']
            topic_schema = topic['schema']
            topic_BronzeTblName = topic['BronzeTblName']
            self.topics = appendTo_DF(self.topics,{'topic_name':topic_name,'topic_schema':topic_schema,'topic_BronzeTblName':topic_BronzeTblName})

    def GenerateDLTScript(self):
        feed_path = self.dict_Feed['feed_path']
        topics = 
        generate_dlt_script(feed_path,)


In [0]:

from pyspark.sql import functions as F

@dlt.view
def raw_feed():
    return (
        spark.readStream
        .format('cloudFiles')
        .option('cloudFiles.format','json')
        .load(feed_batch)
        .select(
            F.col('topic')
            ,F.col('key').cast('string')
            ,F.col('value').cast('string')
            ,(F.col('timestamp')/1000).cast('timestamp').alias('create_ts')
            ,F.input_file_name().alias('source_file')
            ,F.current_timestamp().alias('insert_ts')
        )
    )

@dlt.table(name='ct_bookstore.db_bronze.tbl_books')
def books():
    return (
        dlt.read("raw_feed")
        .filter(F.col('topic') == 'books')
        .withColumn('v', F.from_json(F.col('value'), books_schema))
        .select('key', 'create_ts', 'source_file', 'insert_ts', 'v.*')
    )

@dlt.table(name="tbl_customers")
def customers():
    return (
        dlt.read("raw_feed")
        .filter(F.col('topic') == 'customers')
        .withColumn('v', F.from_json(F.col('value'), customers_schema))
        .select('key', 'create_ts', 'source_file', 'insert_ts', 'v.*')
    )

@dlt.table(name="tbl_orders")
def orders():
    return (
        dlt.read("raw_feed")
        .filter(F.col('topic') == 'orders')
        .withColumn('v', F.from_json(F.col('value'), orders_schema))
        .select('key', 'create_ts', 'source_file', 'insert_ts', 'v.*')
    )